In [1]:
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()


True

In [2]:
import os

# 디버깅을 위한 프로젝트명을 기입합니다.
os.environ["LANGCHAIN_PROJECT"] = "RAG TUTORIAL"

# tracing 을 위해서는 아래 코드의 주석을 해제하고 실행합니다.
# os.environ["LANGCHAIN_TRACING_V2"] = true


In [3]:
from langchain.document_loaders import PyPDFLoader

# PDF 파일 로드. 파일의 경로 입력
loader = PyPDFLoader("data/SPRI_AI_Brief_2023년12월호_F.pdf")

# 페이지 별 문서 로드
docs = loader.load()
print(f"문서의 수: {len(docs)}")

# 10번째 페이지의 내용 출력
print(f"\n[페이지내용]\n{docs[10].page_content[:500]}")
print(f"\n[metadata]\n{docs[10].metadata}\n")


문서의 수: 23

[페이지내용]
SPRi AI Brief |  
2023-12 월호
8코히어 , 데이터 투명성 확보를 위한 데이터 출처 탐색기 공개
n코히어와 12개 기관이  광범위한 데이터셋에 대한 감사를 통해 원본 데이터 출처, 재라이선스 상태, 
작성자 등 다양한 정보를 제공하는 ‘데이터 출처 탐색기 ’ 플랫폼을 출시
n대화형 플랫폼을 통해 개발자는 데이터셋의 라이선스 상태를 쉽게 파악할 수 있으며 데이터셋의 
구성과 계보도 추적 가능KEY Contents
£데이터 출처 탐색기 , 광범위한 데이터셋 정보 제공을 통해 데이터 투명성 향상
nAI 기업 코히어 (Cohere) 가 매사추세츠 공과⼤(MIT), 하버드 ⼤ 로스쿨 , 카네기멜론 ⼤ 등 12개 기관과  
함께 2023 년 10월 25일 ‘데이터 출처 탐색기 (Data Provenance Explorer)’ 플랫폼을 공개
∙AI 모델 훈련에 사용되는 데이터셋의 불분명한 출처로 인해 데이터 투명성이 확보되지 않아 다양한 
법적·윤리적 문제가 발생
∙이에 연구

[metadata]
{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pdf', 'page': 10}



# 1단계 문서 로드(Load Document)

## 특정 폴더에 있는 모든 pdf문서를 로드한다.

In [4]:
from langchain_community.document_loaders import PyPDFDirectoryLoader

loader = PyPDFDirectoryLoader('./data/')
docs  = loader.load()

print(f"문서의 수: {len(docs)}")

# 10번째 페이지의 내용 출력
print(f"\n[페이지내용]\n{docs[10].page_content[:500]}")
print(f"\n[metadata]\n{docs[10].metadata}\n")


문서의 수: 123

[페이지내용]
21 20
ESG Special Report 2023
투자 검토 단계
Pre-Acquisition (인수 전)
01포트폴리오 ESG 관리 체계
장기적 관점에서 기업가치 제고를 실현하기 위해 핵심자산인 
투자 포트폴리오의 경제적 가치와 함께 ESG 가치를 
통합적으로 관리하기 위한 체계를 구축하고 있습니다.
투자 검토 시점부터 인수 후, 회수 시점까지 투자
Life Cycle에 걸쳐 적용되는 체계적인 ESG 관리를 
기반으로 내부적으로는 ESG를 고려한 합리적인 투자의사 
결정을 이행하고, 시장에서는 포트폴리오의 기업가치가 
시장에서 제대로 평가받으며 나아가 사회·환경에 미치는 
파급력을 높일 수 있도록 노력하겠습니다.포트폴리오 ESG 관리 원칙
SK주식회사 투자회사
기업가치 관점의
ESG 중점관리 
항목 도출자사 ESG 
관리전략
ESG 성과 
데이터 관리기업가치와
ESG 성과 
연계성 분석포트폴리오 
ESG 관리전략 
Upgrade성장단계
산업특성
ESG Divestment 전략 

[metadata]
{'source': 'data\\000660_SK_2023.pdf', 'page': 10}



# 2단계 문서 분할(split documents)

## 로드한 문서를 청크로 나누고 인덱싱한다.

### RecursiveCharacterTextSpliter를 사용하여 분할

In [5]:
# langchain 패키지에서 RecursiveCharacterTextSplitter 클래스를 가져옵니다.

# 텍스트를 재귀적으로 분할하는 기능을 제공
# chunk_size는 분할할 청크의 크기
# chunk_overlap으로 인접 청크 간의 겹침 크기
# length_function으로 청크의 길이를 계산하는 함수
# is_separator_regex로 구분자가 정규 표현식인지 여부를 지정하는 매개변수를 받음
from langchain.text_splitter import RecursiveCharacterTextSplitter

recursive_text_splitter = RecursiveCharacterTextSplitter(
    # 정말 작은 청크 크기를 설정합니다.
    separators=["\n\n", "\n", " ", ""],
    chunk_size=512,
    chunk_overlap=128,
    length_function=len,
    is_separator_regex=False,
)


In [6]:
chunked_docs = recursive_text_splitter.split_documents(docs)

# 3단계 임베딩

## huggingface에 있는 임베딩모델을 사용해서 문서를 임베딩

In [12]:
from langchain_community.embeddings import HuggingFaceEmbeddings


# Embeddings Configuration
EMBEDDINGS_MODEL_NAME = 'jhgan/ko-sroberta-nli'
EMBEDDINGS_MODEL_KWARGS = {'device': 'cpu'}
EMBEDDINGS_ENCODE_KWARGS = {'normalize_embeddings': True}

def get_embeddings():
    """Get embeddings for the vectorstore."""
    return HuggingFaceEmbeddings(
        model_name=EMBEDDINGS_MODEL_NAME,
        model_kwargs=EMBEDDINGS_MODEL_KWARGS,
        encode_kwargs=EMBEDDINGS_ENCODE_KWARGS
    )

# 4단계 임베딩한 문서를 벡터 DB에 저장

## 벡터DB는 크게 FAISS or chroma 중 사용

In [13]:
from langchain_community.vectorstores import FAISS

huggingFaceEmbedding = get_embeddings()

# 단계 3: 임베딩 & 벡터스토어 생성(Create Vectorstore)
# 벡터스토어를 생성합니다.
# FAISS 사용
vectorstore = FAISS.from_documents(
    documents=chunked_docs, embedding=get_embeddings()
)


In [ ]:
from langchain_community.vectorstores import Chroma

# Chroma DB 적용
vectorstore = Chroma.from_documents(documents=chunked_docs, embedding=get_embeddings())


# 5단계 Retriever 생성

### 구조화되지 않은 쿼리가 주어지면 문서를 반환하는 인터페이스
### 생성된 VectorStore에 as_retriever()로 가져와서 Retriever를 생성

In [58]:
query = "구글 딥마인드에서 어떤 정책을 발표했는지 알려줘"

retriever = vectorstore.as_retriever()
search_result = retriever.get_relevant_documents(query)

In [68]:
for docs in search_result:
    print(f"{docs.metadata['source']} | {docs.metadata['page']}")
    print()
    print(docs.page_content)
    print()

data\SPRI_AI_Brief_2023년12월호_F.pdf | 18

16구글 딥마인드 , 범용 AI 모델의 기능과 동작에 대한 분류 체계 발표
n구글 딥마인드 연구진이 성능과 범용성 , 자율성을 기준으로 범용 AI(AGI) 의 수준을 
0~5단계까지 총 6단계로 구분한 프레임워크를 공개
n현재 AGI는 단백질 구조를 예측하는 알파폴드와 같은 특정 용도에서는 5단계 수준을 달성했지만 
광범위하게 활용될 수 있는 범용에서는 1단계 수준에 머물러 있음KEY Contents
£챗GPT와 구글 바드와 같은 AI 챗봇은 범용 AI 1단계 수준
n구글 딥마인드 연구진은 2023 년 11월 4일 범용 AI(Artificial General Intelligence, AGI) 모델을 용도와 
성능에 따라 분류하는 프레임워크를 제시한 논문을 발표
∙프레임워크의 목적은 AGI의 성능, 범용성 , 자율성 수준을 정의하여 모델 간 비교와 위험 평가, AGI 
달성까지의 진행 상황을 측정할 수 있는 공통 기준을 제공하기 위함

data\SPRI_AI_Brief_2023년12월호_F.pdf | 13

1. 정책/법제  2. 기업/산업 3. 기술/연구  4. 인력/교육
구글, 앤스로픽에 20억 달러 투자로 생성 AI 협력 강화 
n구글이 앤스로픽에 최대 20억 달러 투자에 합의하고 5억 달러를 우선 투자했으며 , 앤스로픽은 
구글과 클라우드 서비스 사용 계약도 체결
n3대 클라우드 사업자인 구글, 마이크로소프트 , 아마존은 차세대 AI 모델의 대표 기업인 
앤스로픽 및 오픈AI와 협력을 확대하는 추세KEY Contents
£구글, 앤스로픽에 최대 20억 달러 투자 합의 및 클라우드 서비스 제공
n구글이 2023 년 10월 27일 앤스로픽에 최대 20억 달러를 투자하기로 합의했으며 , 이 중 5억 
달러를 우선 투자하고 향후 15억 달러를 추가로 투자할 방침
∙구글은 2023 년 2월 앤스로픽에 이미 5억 5,000 만 달러를 투자한 바 있으며 , 아마존도 지난 9월 
앤스로픽에 최대 

# 6단계 프롬프트 생성

### tip: retriever에서 도출한 결과에서 중요한 ㅈ어보가 누락된다면 retriever의 로직을 수정해야함

### tip: 도출한 결과가 많은 정보를 포함하고 있지만 llm이 그 중에서 중요한 정보를 찾지 못하거나 원하는 형태로 출력하지 않는다면 프롬프트를 수정해야 합니다.

In [28]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [ ]:
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from my_messages import stream_response

# Ollama 모델을 불러옵니다.
llm = ChatOllama(model="llama3.1")

# 프롬프트
prompt = ChatPromptTemplate.from_template("{topic} 에 대하여 간략히 설명해 줘.")

# 체인 생성
chain = prompt | llm | StrOutputParser()

# 간결성을 위해 응답은 터미널에 출력됩니다.
answer = chain.stream({"topic": "deep learning"})


### RAG 템플릿 실험

In [2]:
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Embeddings Configuration
EMBEDDINGS_MODEL_NAME = 'jhgan/ko-sroberta-nli'
EMBEDDINGS_MODEL_KWARGS = {'device': 'cpu'}
EMBEDDINGS_ENCODE_KWARGS = {'normalize_embeddings': True}

def get_embeddings():
    """Get embeddings for the vectorstore."""
    return HuggingFaceEmbeddings(
        model_name=EMBEDDINGS_MODEL_NAME,
        model_kwargs=EMBEDDINGS_MODEL_KWARGS,
        encode_kwargs=EMBEDDINGS_ENCODE_KWARGS
    )


def format_docs(docs):
    # 검색한 문서 결과를 하나의 문단으로 합쳐줍니다.
    return "\n\n".join(doc.page_content for doc in docs)


def print_retrieved_docs(docs):
    """Retrieved documents를 출력하는 함수"""
    print("\n=== Retrieved Documents ===")
    for i, doc in enumerate(docs, 1):
        print(f"\nDocument {i}:")
        print(f"{doc.metadata['source']} |  page: {doc.metadata['page']}")
        print(doc.page_content)
        print("-" * 50)

    print("\n\nAnser: \n")
    return docs

def format_docs_with_print(docs):
    """문서를 출력하고 포맷팅하는 함수"""
    print_retrieved_docs(docs)
    return format_docs(docs)

# 단계 1: 문서 로드
loader = PyPDFDirectoryLoader('./kisa_pdf/')
docs  = loader.load()

# 단계 2: 문서 분할
recursive_text_splitter = RecursiveCharacterTextSplitter(
    # 정말 작은 청크 크기를 설정합니다.
    separators=["\n\n", "\n", " ", ""],
    chunk_size=512,
    chunk_overlap=64,
    length_function=len,
    is_separator_regex=False,
)

chunked_docs = recursive_text_splitter.split_documents(docs)

# 단계 3,4: 임베딩 & 벡터 스토어 생성
# FAISS 사용
faiss_vectorstore = FAISS.from_documents(
    documents=chunked_docs, embedding=get_embeddings()
)

# 단계 5: 리트리버 생성
k = 2
retriever = faiss_vectorstore.as_retriever(search_kwargs={"k": k})

# 단계 6: 프롬프트 생성
prompt = hub.pull("rlm/rag-prompt")

# 단계7: 언어모델 생성
# Ollama 모델을 불러옵니다.
llm = ChatOllama(model="llama3.1", temperature=0)

# 단계 8: 체인 생성
rag_chain = (
    {
        "context": retriever | format_docs_with_print,
        "question": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)

c:\Users\SSAFY\miniconda3\envs\py310\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


KeyboardInterrupt: 

In [ ]:
from my_messages import stream_response

# 단계 8: 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
# question = "구글 딥마인드 정책에 대해서 알려주세요"
# question = "미래의 AI 소프트웨어 매출 전망은 어떻게 되나요?"
# question = "YouTube 가 2024년에 의무화 한 것은 무엇인가요?"
# question = "연차는 어떻게 사용할 수 있나요?"
question = "유류비 청구는 어떻게 할 수 있나요? 제가 3박 4일로 부산에 갓다오는데 비용은 대충 어떻게 계산하죠?"

response = rag_chain.stream(question)

# 결과 출력
print(f"[HUMAN]\n{question}\n")
print("===" * 20)
# print(rag_chain["retrieved_docs"])
stream_response(response)

### 대화 내용을 기억하는 기능을 추가

In [75]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.memory import ConversationBufferWindowMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# Embeddings Configuration
EMBEDDINGS_MODEL_NAME = 'jhgan/ko-sroberta-nli'
EMBEDDINGS_MODEL_KWARGS = {'device': 'cpu'}
EMBEDDINGS_ENCODE_KWARGS = {'normalize_embeddings': True}

def get_embeddings():
    """Get embeddings for the vectorstore."""
    return HuggingFaceEmbeddings(
        model_name=EMBEDDINGS_MODEL_NAME,
        model_kwargs=EMBEDDINGS_MODEL_KWARGS,
        encode_kwargs=EMBEDDINGS_ENCODE_KWARGS
    )

def format_docs(docs):
    # 검색한 문서 결과를 하나의 문단으로 합쳐줍니다.
    return "\n\n".join(doc.page_content for doc in docs)


def print_retrieved_docs(docs):
    """Retrieved documents를 출력하는 함수"""
    print("\n=== Retrieved Documents ===")
    for i, doc in enumerate(docs, 1):
        print(f"\nDocument {i}:")
        print(f"{doc.metadata['source']} |  page: {doc.metadata['page']}")
        print(doc.page_content)
        print("-" * 50)

    print("\n\nAnser: \n")
    return docs

def format_docs_with_print(docs):
    """문서를 출력하고 포맷팅하는 함수"""
    print_retrieved_docs(docs)
    return format_docs(docs)

def invoke_chain_with_memory(question: str, chain, memory):
    """메모리를 포함하여 체인을 실행하는 함수"""
    result = chain.invoke({
        "question": question,
        "chat_history": memory.load_memory_variables({})["chat_history"]
    })
    
    # 메모리에 대화 저장
    memory.save_context(
        {"question": question},
        {"answer": result}
    )
    
    return result


In [76]:
# 메모리 컴포넌트 생성
memory = ConversationBufferWindowMemory(
    return_messages=True,
    memory_key="chat_history",
    output_key="answer"
)

# 단계 1: 문서 로드
loader = PyPDFDirectoryLoader('./kisa_pdf/')
docs  = loader.load()

# 단계 2: 문서 분할
recursive_text_splitter = RecursiveCharacterTextSplitter(
    # 정말 작은 청크 크기를 설정합니다.
    separators=["\n\n", "\n", " ", ""],
    chunk_size=512,
    chunk_overlap=64,
    length_function=len,
    is_separator_regex=False,
)

chunked_docs = recursive_text_splitter.split_documents(docs)

# 단계 3,4: 임베딩 & 벡터 스토어 생성
# FAISS 사용
faiss_vectorstore = FAISS.from_documents(
    documents=chunked_docs, embedding=get_embeddings()
)

# 단계 5: 리트리버 생성
k = 2
retriever = faiss_vectorstore.as_retriever(search_kwargs={"k": k})


# 단계 6: 프롬프트 생성
# 새로운 프롬프트 템플릿 생성 (기존 rlm/rag-prompt를 기반으로 수정)
prompt = ChatPromptTemplate.from_messages([
    ("system", '''You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question.
     If you don't know the answer, just say that you don't know.
     Use three sentences maximum and keep the answer concise. Answer in Korean.'''),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "Context: {context}\n\nQuestion: {question}"),
])

# 단계7: 언어모델 생성
# Ollama 모델을 불러옵니다.
llm = ChatOllama(model="llama3.1", temperature=0)

# 단계 8: 체인 생성
rag_chain = (
    {
        "context": retriever | format_docs_with_print,
        "question": RunnablePassthrough(),
        "chat_history": lambda x: memory.load_memory_variables({})["chat_history"],
    }
    | prompt
    | llm
    | StrOutputParser()
)

C:\Users\SSAFY\AppData\Local\Temp\ipykernel_13076\2753456349.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(


In [79]:
from my_messages import stream_response

# 단계 8: 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
# question = "구글 딥마인드 정책에 대해서 알려주세요"
# question = "미래의 AI 소프트웨어 매출 전망은 어떻게 되나요?"
# question = "YouTube 가 2024년에 의무화 한 것은 무엇인가요?"
question = "이전 답변을 영어로 번역해주세요"
response = rag_chain.stream(question)

# 결과 출력
print(f"[HUMAN]\n{question}\n")
print("===" * 20)
# print(rag_chain["retrieved_docs"])
stream_response(response)

[HUMAN]
이전 답변을 영어로 번역해주세요


=== Retrieved Documents ===

Document 1:
data\SPRI_AI_Brief_2023년12월호_F.pdf |  page: 3
∙비자 기준과 인터뷰 절차의 현대화와 간소화로 AI 관련 주요 분야의 전문 지식을 갖춘 외국인들이 미국에서 
공부하고 취업할 수 있도록 지원
☞ 출처 : The White House, Executive Order on the Safe, Secure, and Trustworthy Development and Use of 
Artificial Intelligence (E.O. 14110), 2023.10.30.
--------------------------------------------------

Document 2:
data\000660_SK_2023.pdf |  page: 0
1
Where we are heading    |     How we get there    |     What we are preparing
ESG Special Report
2023 NAVIGATING 
UNCERTAINTIES TO ENSURE  
SUSTAINABLE 
GROWTH
--------------------------------------------------


Anser: 

The previous answer in English translation is:

"The modernization and simplification of visa standards and interview procedures allow AI-related major fields experts from foreign countries to study and work in the United States."